# MAS - project 

# IDA*

In [10]:
import random
import numpy as np
import itertools
import time
import amd_project_functions as amd

In [11]:
class IDA_SearchAlgorithm:
    def __init__(self, O, Theta, p, o_XY, v_XY, g, u, L_initial_limit, CB = None, L_reduction = 0.1):
        """
        Initialize the IDA object.
        """
        self.O = O
        self.n = len(O) - 1 
        self.Theta = Theta
        self.p = p
        self.o_XY = o_XY
        self.v_XY = v_XY
        self.g = g
        self.u = u
        self.L = L_initial_limit
        self.CB = CB
        self.HF = -np.inf # different from the serach alg
        self.L_reduction = L_reduction # different from the serach alg

    def search2(self, X, Y, v, d):
        """
        Perform the SEARCH2 algorithm.
        """
        if d == self.n + 1: # end of the tree
            self.CB = X # update the current best X solution (here there is not the check that the best solution is better than the current one. No problem, it there is one before calling the function search)
            self.L = v # update the current best value of X 
        else:            
            if self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search2(amd.union_listOflists_list(X,self.O[d]), Y, self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u), d + 1)
            elif self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.HF:
                self.HF = self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u)

            if self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search2(X,amd.union_listOflists_list(Y,self.O[d]), self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u), d + 1)          
            elif self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.HF:
                self.HF = self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) 

In [12]:
def ida(O, Theta, p, o_XY, v_XY, g, u, L_initial_limit ,L_reduction = 0.1):
    start_time = time.time()  # Record the start time

    CB = None # initilize the current best solution
    L = L_initial_limit
    
    while CB == None:
        HF = -np.inf
        IDA_SearchAlgorithm_instance = IDA_SearchAlgorithm(O, Theta, p, o_XY, v_XY, g, u, L, CB, L_reduction)
        IDA_SearchAlgorithm_instance.search2([], [], 0, 1)
        CB = IDA_SearchAlgorithm_instance.CB
        HF = IDA_SearchAlgorithm_instance.HF
        L = min(HF, L*(1-L_reduction))

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    return CB, round(elapsed_time,2)

# Applications

## 1 - No IR

In [13]:
# An instance
O = [[i] for i in range(30)]
Theta = [i for i in range(10)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=0, maximum_value=100)
u_instance = amd.u_class(Theta, O, minimum_value=0, maximum_value=100) 

L_initial_limit = 80
L_reduction = 0.3


In [14]:
def ida_find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance, L_initial_limit, L_reduction = 0.1):
    CB, elapsed_time = ida(O, Theta, p, o_XY, v_XY, g_instance.g, u_instance.u, L_initial_limit=L_initial_limit, L_reduction = L_reduction)
    model_vX_value = amd.v_X(CB, Theta, p, g_instance.g, u_instance.u)

    semi_random_CB = random.sample(O, len(CB))
    semi_random_vX_value = amd.v_X(semi_random_CB, Theta, p, g_instance.g, u_instance.u)
    if model_vX_value != 0:
        model_vs_rand = round((model_vX_value-semi_random_vX_value)/model_vX_value * 100,2)
    else:
        model_vs_rand = None

    return CB, round(model_vX_value,2), elapsed_time, semi_random_CB, round(semi_random_vX_value,2), model_vs_rand

In [15]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction)

print("Check IR for the model", amd.check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  218.19
Model X = [[2], [11], [17], [23], [29], [3], [9], [12], [18], [21], [24], [4], [27], [7], [10], [16], [13], [25], [22]]
Semi random model X =  [[19], [12], [2], [17], [16], [28], [21], [18], [11], [29], [14], [10], [0], [24], [20], [23], [1], [26], [15]]

Model vX value  89.4
Semi random model vX value  77.0
Relative improvement of the model  13.87  %


In [16]:
# Check the model on a single type

def check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB):
    outcomeForAllThetas_model = [amd.o_X(theta = theta, X = CB, g = g_instance.g, u = u_instance.u) for theta in Theta]
    gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
    uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

    outcomeForAllThetas_semiRandom = [amd.o_X(theta = theta, X = semi_random_CB, g = g_instance.g, u = u_instance.u) for theta in Theta]
    gForAllOutcomes_semiRandom = [g_instance.g(theta=theta, o=o_semiRandom) for theta, o_semiRandom in zip(Theta, outcomeForAllThetas_semiRandom)]
    uForAllOutcomes_semiRandom = [u_instance.u(theta=theta, o=o_semiRandom) for theta, o_semiRandom in zip(Theta, outcomeForAllThetas_semiRandom)]

    return gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom



gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

         g   u
model    89.4 96.0
semi rnd 77.0 97.5


## 2 - IR

In [17]:
# An instance
O = [[i] for i in range(20)]
Theta = [i for i in range(10)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=-50, maximum_value=50)
u_instance = amd.u_class(Theta, O, minimum_value=-50, maximum_value=50) 

L_initial_limit = 35
L_reduction = 0.1


In [18]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand  = ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction)

print("Check IR for the model", amd.check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  4.59
Model X = [[18], [2], [15], [16], [13], [8], [3], [9]]
Semi random model X =  [[7], [8], [17], [5], [9], [0], [16], [18]]

Model vX value  31.5
Semi random model vX value  3.2
Relative improvement of the model  89.84  %


In [19]:
# Check g and u
gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

         g   u
model    31.5 36.9
semi rnd 3.2 35.7


## 3 - Bartering

In [20]:
# An instance
number_of_items = 10
number_of_outcomes = 25
number_of_types = 10
minimum_value = 0 # try -5
maximum_value = 10 # try 5

L_initial_limit = 60
L_reduction = 0.1

all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)
u_instance = amd.u_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value) 


In [21]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction)

print("Check IR for the model", amd.check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  2.37
Model X = [[1, 0, 0, 0, 0, 0, 1, 1, 0, 0]]
Semi random model X =  [(0, 0, 0, 1, 0, 0, 0, 0, 0, 1)]

Model vX value  31.0
Semi random model vX value  20.0
Relative improvement of the model  35.48  %


In [22]:
# Check g and u
gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

         g   u
model    31.0 13.9
semi rnd 20.0 11.6


# Extra things

## Another utility type for the batering problem

In [23]:
# An instance
number_of_items = 10
number_of_outcomes = 100
number_of_types = 5
minimum_value = 0 # with [0,10] the IR constraint is mainly = False 
maximum_value = 10

L_initial_limit = 60
L_reduction = 0.1

all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
initial_goods = random.choice(O)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods)
u_instance = amd.u_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods) 



In [24]:
CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = ida_find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance, L_initial_limit, L_reduction)

print("Check IR for the model", amd.check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

Check IR for the model False
Check IR for the semi-rnd True

Elapsed time =  8.57
Model X = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
Semi random model X =  [(1, 0, 1, 1, 0, 1, 0, 1, 1, 0)]

Model vX value  35.0
Semi random model vX value  1.0
Relative improvement of the model  97.14  %


In [25]:
# Check g and u
gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))


         g   u
model    35.0 -23.8
semi rnd 1.0 -2.4
